<a href="https://colab.research.google.com/github/hardikroy24/DAARKGPT/blob/main/Qwen3_TTS_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎤 Qwen3-TTS - Complete TTS Suite

**Features:**
- ✅ Text-to-Speech (TTS) with Custom Voices
- ✅ Voice Cloning (3-second rapid clone)
- ✅ Voice Design (Create voices from natural language descriptions)

**Requirements:** Google Colab with GPU (T4 or better)

---

## 1. 🔧 Environment Setup

In [1]:
# Check GPU availability
!nvidia-smi

Sun Feb 15 22:22:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install qwen-tts package and dependencies
!pip install -U qwen-tts gradio soundfile numpy

# Install FlashAttention 2 for better performance (optional but recommended)
!pip install -U flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 66.2 MB/s eta 0:00:00
  Created wheel for sox: filename=sox-1.5.0-py3-none-any.whl size=40036 sha256=1a2a6ce4ef63ff1e49aff298df2659993758e67d7c9fdd6df746eb4e112c2795
  Stored in directory: /root/.cache/pip/wheels/8c/c7/e7/baea1f7e79b9eb53addc81cc9b827424f4a7d8c9cc

In [3]:
# Verify installation
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

PyTorch version: 2.9.0+cu128
CUDA available: True
GPU: Tesla T4
GPU Memory: 14.6 GB


## 2. 📦 Load Models

We'll load all three models:
- **CustomVoice**: For TTS with preset voices
- **Base**: For voice cloning
- **VoiceDesign**: For creating voices from descriptions

In [4]:
import torch
import soundfile as sf
from qwen_tts import Qwen3TTSModel
import numpy as np
import tempfile
import os

# Determine dtype and attention implementation based on GPU capability
if torch.cuda.is_available():
    device = "cuda:0"
    dtype = torch.bfloat16
    try:
        attn_impl = "flash_attention_2"
    except:
        attn_impl = "sdpa"
else:
    device = "cpu"
    dtype = torch.float32
    attn_impl = "eager"

print(f"Device: {device}")
print(f"Dtype: {dtype}")
print(f"Attention: {attn_impl}")


    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


Device: cuda:0
Dtype: torch.bfloat16
Attention: flash_attention_2


In [5]:
# Load CustomVoice model (for TTS)
print("Loading CustomVoice model...")
custom_voice_model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice",
    device_map=device,
    dtype=dtype,
    attn_implementation=attn_impl,
)
print("✅ CustomVoice model loaded!")

Loading CustomVoice model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2 without specifying a torch dtype. This might lead to unexpected behaviour


...

preprocessor_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

✅ CustomVoice model loaded!


In [6]:
# Load Base model (for voice cloning)
print("Loading Base model for voice cloning...")
clone_model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-Base",
    device_map=device,
    dtype=dtype,
    attn_implementation=attn_impl,
)
print("✅ Base model loaded!")

Loading Base model for voice cloning...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

✅ Base model loaded!


In [7]:
# Load VoiceDesign model
print("Loading VoiceDesign model...")
voice_design_model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign",
    device_map=device,
    dtype=dtype,
    attn_implementation=attn_impl,
)
print("✅ VoiceDesign model loaded!")

Loading VoiceDesign model...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

preprocessor_config.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

✅ VoiceDesign model loaded!


## 3. 🎛️ Launch Gradio Web UI

This comprehensive UI includes all three features:
- **TTS Tab**: Text-to-Speech with preset voices and emotion control
- **Voice Clone Tab**: Clone any voice from a short audio sample
- **Voice Design Tab**: Create custom voices using natural language descriptions

In [ ]:
import gradio as gr

# Speaker information
SPEAKERS = {
    "Vivian": "Bright, slightly edgy young female voice (Chinese native)",
    "Serena": "Warm, gentle young female voice (Chinese native)",
    "Uncle_Fu": "Seasoned male voice with a low, mellow timbre (Chinese native)",
    "Dylan": "Youthful Beijing male voice with clear, natural timbre (Chinese - Beijing Dialect)",
    "Eric": "Lively Chengdu male voice with slightly husky brightness (Chinese - Sichuan Dialect)",
    "Ryan": "Dynamic male voice with strong rhythmic drive (English native)",
    "Aiden": "Sunny American male voice with clear midrange (English native)",
    "Ono_Anna": "Playful Japanese female voice with light, nimble timbre (Japanese native)",
    "Sohee": "Warm Korean female voice with rich emotion (Korean native)"
}

LANGUAGES = ["Auto", "Chinese", "English", "Japanese", "Korean", "German", "French", "Russian", "Portuguese", "Spanish", "Italian"]

# TTS function
def generate_tts(text, speaker, language, instruct):
    if not text.strip():
        return None, "Please enter text to synthesize."

    try:
        lang = None if language == "Auto" else language
        wavs, sr = custom_voice_model.generate_custom_voice(
            text=text,
            language=lang,
            speaker=speaker,
            instruct=instruct if instruct.strip() else None,
        )

        # Save to temp file
        output_path = tempfile.mktemp(suffix=".wav")
        sf.write(output_path, wavs[0], sr)
        return output_path, f"✅ Generated successfully! Speaker: {speaker}"
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

# Voice Clone function
def clone_voice(text, ref_audio, ref_text, language):
    if not text.strip():
        return None, "Please enter text to synthesize."
    if ref_audio is None:
        return None, "Please upload a reference audio file."

    try:
        lang = None if language == "Auto" else language

        # Handle reference audio
        if isinstance(ref_audio, tuple):
            # Gradio returns (sample_rate, audio_array)
            sr_ref, audio_ref = ref_audio
            ref_audio_input = (audio_ref, sr_ref)
        else:
            ref_audio_input = ref_audio

        wavs, sr = clone_model.generate_voice_clone(
            text=text,
            language=lang,
            ref_audio=ref_audio_input,
            ref_text=ref_text if ref_text.strip() else None,
        )

        output_path = tempfile.mktemp(suffix=".wav")
        sf.write(output_path, wavs[0], sr)
        return output_path, "✅ Voice cloned successfully!"
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

# Voice Design function
def design_voice(text, voice_description, language):
    if not text.strip():
        return None, "Please enter text to synthesize."
    if not voice_description.strip():
        return None, "Please provide a voice description."

    try:
        lang = None if language == "Auto" else language

        wavs, sr = voice_design_model.generate_voice_design(
            text=text,
            language=lang,
            instruct=voice_description,
        )

        output_path = tempfile.mktemp(suffix=".wav")
        sf.write(output_path, wavs[0], sr)
        return output_path, "✅ Voice designed successfully!"
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

# Build Gradio Interface
with gr.Blocks(title="Qwen3-TTS Suite", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎤 Qwen3-TTS Complete Suite

    Powerful speech synthesis with **TTS**, **Voice Cloning**, and **Voice Design** capabilities.

    Supports 10 languages: Chinese, English, Japanese, Korean, German, French, Russian, Portuguese, Spanish, Italian
    """)

    with gr.Tabs():
        # Tab 1: TTS
        with gr.TabItem("🎙️ Text-to-Speech"):
            gr.Markdown("### Generate speech using preset premium voices")

            with gr.Row():
                with gr.Column():
                    tts_text = gr.Textbox(
                        label="Text to Synthesize",
                        placeholder="Enter the text you want to convert to speech...",
                        lines=4
                    )
                    tts_speaker = gr.Dropdown(
                        choices=list(SPEAKERS.keys()),
                        value="Vivian",
                        label="Speaker"
                    )
                    tts_speaker_info = gr.Markdown(f"*{SPEAKERS['Vivian']}*")
                    tts_language = gr.Dropdown(
                        choices=LANGUAGES,
                        value="Auto",
                        label="Language"
                    )
                    tts_instruct = gr.Textbox(
                        label="Emotion/Style Instruction (Optional)",
                        placeholder="e.g., 'Speak happily', '用愤怒的语气说', 'Very excited'",
                        lines=2
                    )
                    tts_btn = gr.Button("🎵 Generate Speech", variant="primary")

                with gr.Column():
                    tts_output = gr.Audio(label="Generated Audio", type="filepath")
                    tts_status = gr.Markdown()

            # Update speaker info on selection
            tts_speaker.change(
                fn=lambda s: f"*{SPEAKERS.get(s, '')}*",
                inputs=tts_speaker,
                outputs=tts_speaker_info
            )

            tts_btn.click(
                fn=generate_tts,
                inputs=[tts_text, tts_speaker, tts_language, tts_instruct],
                outputs=[tts_output, tts_status]
            )

            # Examples
            gr.Examples(
                examples=[
                    ["其实我真的有发现，我是一个特别善于观察别人情绪的人。", "Vivian", "Chinese", "用特别愤怒的语气说"],
                    ["Hello! Welcome to Qwen3-TTS, the most powerful speech synthesis system.", "Ryan", "English", "Very enthusiastic"],
                    ["おはようございます！今日はいい天気ですね。", "Ono_Anna", "Japanese", ""],
                    ["안녕하세요, 만나서 반갑습니다!", "Sohee", "Korean", ""],
                ],
                inputs=[tts_text, tts_speaker, tts_language, tts_instruct]
            )

        # Tab 2: Voice Clone
        with gr.TabItem("🎭 Voice Cloning"):
            gr.Markdown("""
            ### Clone any voice from a short audio sample

            Upload a 3+ second audio clip of the voice you want to clone, along with its transcript for best results.
            """)

            with gr.Row():
                with gr.Column():
                    clone_ref_audio = gr.Audio(
                        label="Reference Audio (3+ seconds recommended)",
                        type="filepath"
                    )
                    clone_ref_text = gr.Textbox(
                        label="Reference Audio Transcript (Optional but recommended)",
                        placeholder="Enter what is being said in the reference audio...",
                        lines=2
                    )
                    clone_text = gr.Textbox(
                        label="Text to Synthesize",
                        placeholder="Enter the text you want the cloned voice to say...",
                        lines=4
                    )
                    clone_language = gr.Dropdown(
                        choices=LANGUAGES,
                        value="Auto",
                        label="Output Language"
                    )
                    clone_btn = gr.Button("🎭 Clone Voice", variant="primary")

                with gr.Column():
                    clone_output = gr.Audio(label="Cloned Voice Output", type="filepath")
                    clone_status = gr.Markdown()

            clone_btn.click(
                fn=clone_voice,
                inputs=[clone_text, clone_ref_audio, clone_ref_text, clone_language],
                outputs=[clone_output, clone_status]
            )

        # Tab 3: Voice Design
        with gr.TabItem("🎨 Voice Design"):
            gr.Markdown("""
            ### Create custom voices from natural language descriptions

            Describe the voice you want (age, gender, tone, emotion, accent, etc.) and let the AI generate it!
            """)

            with gr.Row():
                with gr.Column():
                    design_text = gr.Textbox(
                        label="Text to Synthesize",
                        placeholder="Enter the text you want to convert to speech...",
                        lines=4
                    )
                    design_description = gr.Textbox(
                        label="Voice Description",
                        placeholder="Describe the voice (e.g., 'A warm, friendly elderly woman with a slight British accent')",
                        lines=4
                    )
                    design_language = gr.Dropdown(
                        choices=LANGUAGES,
                        value="Auto",
                        label="Language"
                    )
                    design_btn = gr.Button("🎨 Design Voice", variant="primary")

                with gr.Column():
                    design_output = gr.Audio(label="Designed Voice Output", type="filepath")
                    design_status = gr.Markdown()

            design_btn.click(
                fn=design_voice,
                inputs=[design_text, design_description, design_language],
                outputs=[design_output, design_status]
            )

            # Examples
            gr.Examples(
                examples=[
                    ["哥哥，你回来啦，人家等了你好久好久了，要抱抱！", "体现撒娇稚嫩的萝莉女声，音调偏高且起伏明显，营造出黏人、做作又刻意卖萌的听觉效果。", "Chinese"],
                    ["It's in the top drawer... wait, it's empty? No way, that's impossible!", "Speak in an incredulous tone, with a hint of panic beginning to creep into your voice.", "English"],
                    ["Welcome to our company. I'm here to help you with any questions.", "Professional, confident female voice, mid-30s, warm but businesslike tone.", "English"],
                    ["Once upon a time, in a land far away, there lived a brave knight.", "Male, 17 years old, tenor range, gaining confidence - deeper breath support now, though vowels still tighten when nervous.", "English"],
                ],
                inputs=[design_text, design_description, design_language]
            )

    gr.Markdown("""
    ---
    **Tips:**
    - For best TTS results, match the speaker's native language
    - Voice cloning works best with clear, 3-10 second audio samples
    - Be creative with voice design descriptions - specify age, gender, emotion, accent, and speaking style!

    *Powered by [Qwen3-TTS](https://github.com/QwenLM/Qwen3-TTS)*
    """)

# Launch with public link for Colab
demo.launch(share=True, debug=True)

/tmp/ipython-input-3777454135.py:93: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(title="Qwen3-TTS Suite", theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d0816a0c987f3a7e25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2150 for open-end generation.


## 4. 📝 Quick Usage Examples (Without UI)

Run these cells for quick testing without the Gradio interface.

In [ ]:
# Quick TTS Example
from IPython.display import Audio, display

text = "Hello! Welcome to Qwen3-TTS. This is a demonstration of high-quality speech synthesis."
wavs, sr = custom_voice_model.generate_custom_voice(
    text=text,
    language="English",
    speaker="Ryan",
    instruct="Speak enthusiastically"
)

print("🎵 TTS Output:")
display(Audio(wavs[0], rate=sr))

In [ ]:
# Quick Voice Design Example
text = "I'm so excited to meet you! This is going to be amazing!"
description = "Young energetic female voice, early 20s, bubbly and enthusiastic, with slight uptalk patterns"

wavs, sr = voice_design_model.generate_voice_design(
    text=text,
    language="English",
    instruct=description
)

print("🎨 Voice Design Output:")
display(Audio(wavs[0], rate=sr))

## 5. 🔄 Alternative: Use Built-in Demo Command

If you prefer, you can use the built-in Qwen-TTS demo command to launch individual model interfaces.

In [ ]:
# Option 1: Launch CustomVoice demo
# !qwen-tts-demo Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice --ip 0.0.0.0 --port 7860

# Option 2: Launch VoiceDesign demo
# !qwen-tts-demo Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign --ip 0.0.0.0 --port 7860

# Option 3: Launch Base (voice cloning) demo
# !qwen-tts-demo Qwen/Qwen3-TTS-12Hz-1.7B-Base --ip 0.0.0.0 --port 7860

## 📌 Notes

### Supported Speakers (CustomVoice Model)

| Speaker | Voice Description | Native Language |
|---------|------------------|----------------|
| Vivian | Bright, slightly edgy young female voice | Chinese |
| Serena | Warm, gentle young female voice | Chinese |
| Uncle_Fu | Seasoned male voice with low, mellow timbre | Chinese |
| Dylan | Youthful Beijing male voice | Chinese (Beijing) |
| Eric | Lively Chengdu male voice | Chinese (Sichuan) |
| Ryan | Dynamic male voice with strong rhythm | English |
| Aiden | Sunny American male voice | English |
| Ono_Anna | Playful Japanese female voice | Japanese |
| Sohee | Warm Korean female voice | Korean |

### Supported Languages
Chinese, English, Japanese, Korean, German, French, Russian, Portuguese, Spanish, Italian

### Memory Requirements
- 1.7B models: ~8-10 GB GPU memory
- 0.6B models: ~4-6 GB GPU memory (use these if you have limited VRAM)

To use smaller models, change the model names to:
- `Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice`
- `Qwen/Qwen3-TTS-12Hz-0.6B-Base`